# Libraries installation on the current python environment

In [26]:
#%reset -f
try:
    import networkx 
except:
    %pip install networkx==3.1
    %pip install numpy==1.23.5
    %pip install pandas==2.0.3
    %pip install pillow==10.0.0
    %pip install scikit-learn==1.3.0
    %pip install scipy==1.11
    %pip install matplotlib==3.7.1
    %pip isntall matplotlib-inline==0.1.6

# Pre-processing original data

In [27]:
import csv
def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break

In [28]:
sampleAnno = 0

## Reading from dataset

In [29]:
import os
import pandas as pd
ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
with open(ANNOTATIONS_PATH+'/annotationsVSingle.txt','r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    print(len(annotations))

if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
    folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
else:
    folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
print(sampleData,startSec,endSec)
with open(sampleData,'r') as file:
    numMarkers, columnNames, readerBuffer = custom_header_reader(file)
    data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))
if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
    raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
else:
    print("\x1b[32mSTART AND END SECONDS CHECK CORRECT!\x1b[0m")
data.head()

60
data/raw/2016-03-21/t_003.tsv 37.0 40.5
START AND END SECONDS CHECK CORRECT!


,Frame,Time,SMPTE,ARIEL X,ARIEL Y,ARIEL Z,LFHD X,LFHD Y,LFHD Z,RFHD X,...,LBHD Z,LPLM X,LPLM Y,LPLM Z,RPLM X,RPLM Y,RPLM Z,RFTHI X,RFTHI Y,RFTHI Z
0,3553,37.00000,00:35:14:05,942.102,1626.787,-104.007,881.552,1589.723,-39.112,897.100,...,-18.856,797.524,664.863,123.282,862.068,645.962,-303.352,861.573,663.841,-282.063
1,3554,37.01042,00:35:14:05,942.115,1626.776,-104.065,881.567,1589.719,-39.162,897.084,...,-18.886,797.451,664.943,123.405,861.410,645.997,-303.219,861.394,663.881,-281.903
2,3555,37.02083,00:35:14:05,942.118,1626.727,-104.136,881.576,1589.731,-39.222,897.071,...,-18.943,797.365,665.032,123.543,860.476,646.005,-302.976,861.115,663.934,-281.656
3,3556,37.03125,00:35:14:06,942.139,1626.698,-104.192,881.582,1589.750,-39.274,897.070,...,-18.967,797.235,665.123,123.660,859.635,646.003,-302.756,860.837,663.944,-281.388
4,3557,37.04167,00:35:14:06,942.133,1626.686,-104.260,881.584,1589.759,-39.315,897.049,...,-18.998,797.093,665.214,123.780,858.658,645.999,-302.496,860.491,663.934,-281.086


## Standardizing column names

In [30]:
print(len(data))
data.iloc[:,3::3].head()

337


,ARIEL X,LFHD X,RFHD X,C7 X,T5 X,T10 X,BWT X,CLAV X,STRN X,LBSH X,...,RMT1 X,RMT5 X,RMID X,LFRM X,LFSH X,RBHD X,LBHD X,LPLM X,RPLM X,RFTHI X
0,942.102,881.552,897.100,1009.001,1034.459,1018.896,1024.679,765.815,794.537,1032.174,...,852.668,864.226,896.682,871.530,829.644,987.325,985.903,797.524,862.068,861.573
1,942.115,881.567,897.084,1008.936,1034.377,1018.779,1024.559,765.516,794.414,1032.213,...,852.673,864.230,895.634,871.545,829.574,987.291,985.891,797.451,861.410,861.394
2,942.118,881.576,897.071,1008.883,1034.280,1018.146,1024.435,765.238,794.313,1032.153,...,852.675,864.239,894.756,871.518,829.495,987.269,985.866,797.365,860.476,861.115
3,942.139,881.582,897.070,1008.985,1034.183,1018.045,1024.338,764.974,794.190,1032.119,...,852.680,864.235,893.859,871.507,829.428,987.192,985.833,797.235,859.635,860.837
4,942.133,881.584,897.049,1008.949,1034.101,1017.950,1024.751,764.720,794.046,1032.116,...,852.662,864.230,893.315,871.489,829.325,987.156,985.833,797.093,858.658,860.491


In [31]:
[colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

['ARIEL',
 'LFHD',
 'RFHD',
 'C7',
 'T5',
 'T10',
 'BWT',
 'CLAV',
 'STRN',
 'LBSH',
 'LSHO',
 'LFUPA',
 'LBUPA',
 'LIEL',
 'LELB',
 'LOWR',
 'LIWR',
 'RSHO',
 'RBSH',
 'RFUPA',
 'RBUPA',
 'RFSH',
 'RIEL',
 'RELB',
 'RFRM',
 'ROWR',
 'RIWR',
 'LFWT',
 'RFWT',
 'LBWT',
 'RBWT',
 'LFTHI',
 'LBTHI',
 'LKNE',
 'LKNI',
 'LSHN',
 'LANK',
 'LHEL',
 'LTOE',
 'LMT1',
 'LMT5',
 'RBTHI',
 'RKNE',
 'RKNI',
 'RSHN',
 'RANK',
 'RHEL',
 'RTOE',
 'RMT1',
 'RMT5',
 'RMID',
 'LFRM',
 'LFSH',
 'RBHD',
 'LBHD',
 'LPLM',
 'RPLM',
 'RFTHI']

In [32]:
import pandas as pd
if '_A' in data.columns[3] or '_B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
elif '61A' in data.columns[3] or '61B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
if '_01' in data.columns[3]:
    data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
    mappings = pd.read_csv("data/raw/ExtendedMarkersToStandard.csv")
    mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
    data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]
    

In [33]:
list(data.columns)[:3]+list(data.columns)[3::3]

['Frame',
 'Time',
 'SMPTE',
 'ARIEL X',
 'LFHD X',
 'RFHD X',
 'C7 X',
 'T5 X',
 'T10 X',
 'BWT X',
 'CLAV X',
 'STRN X',
 'LBSH X',
 'LSHO X',
 'LFUPA X',
 'LBUPA X',
 'LIEL X',
 'LELB X',
 'LOWR X',
 'LIWR X',
 'RSHO X',
 'RBSH X',
 'RFUPA X',
 'RBUPA X',
 'RFSH X',
 'RIEL X',
 'RELB X',
 'RFRM X',
 'ROWR X',
 'RIWR X',
 'LFWT X',
 'RFWT X',
 'LBWT X',
 'RBWT X',
 'LFTHI X',
 'LBTHI X',
 'LKNE X',
 'LKNI X',
 'LSHN X',
 'LANK X',
 'LHEL X',
 'LTOE X',
 'LMT1 X',
 'LMT5 X',
 'RBTHI X',
 'RKNE X',
 'RKNI X',
 'RSHN X',
 'RANK X',
 'RHEL X',
 'RTOE X',
 'RMT1 X',
 'RMT5 X',
 'RMID X',
 'LFRM X',
 'LFSH X',
 'RBHD X',
 'LBHD X',
 'LPLM X',
 'RPLM X',
 'RFTHI X']

## Working on NaNs

In [34]:
print("Total NaNs")
nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(nans) == 0 else '\x1b[31m'
print(color+str(nans)+'\x1b[0m '+ ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Total NaNs
Series([], dtype: float64) 
No Nans


In [35]:
from itertools import groupby
max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))
print("Max contiguous NaNs")
max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_nans_per_column) == 0 else '\x1b[31m'
print(color+str(max_nans_per_column)+'\x1b[0m ' + ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Max contiguous NaNs
Series([], dtype: int64) 
No Nans


## Working on Zeros

In [36]:
print('Num Zeros')
zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
#list(zeros.index)

Num Zeros
LIEL X    63
LIEL Y    63
LIEL Z    63
LIWR X    25
LIWR Y    25
LIWR Z    25
dtype: int64 
Zeros found!


In [37]:
# THIS IS A FIX FOR MISSING QTM AT DATA 58-59
import numpy as np
if len(zeros.index) > 15:
    for name_col,num_zeros in zeros.items():
        if num_zeros < 50:
            data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
        

In [38]:
from itertools import groupby
max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
print("Max contiguous Zeros")
max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_zeros_per_column) == 0 else '\x1b[31m'
print(color+str(max_zeros_per_column)+'\x1b[0m ' + ('\nNo Zeros' if len(max_zeros_per_column)==0 else '\nZeros found!'))

Max contiguous Zeros
LIEL X    63
LIEL Y    63
LIEL Z    63
LIWR X    25
LIWR Y    25
LIWR Z    25
dtype: int64 
Zeros found!


## Converting to reduced marker set

![fullbodymarkers](../resources/fullmarkerset.png) ![markerstable](../resources/markersetable.png)

In [39]:
def map_reduce_num_markers(reducedMarkerNames:list):
    markersMap = {reducedMarkerNames[0]:   ['LHEL','LMT1','LMT5','LTOE'],
                  reducedMarkerNames[1]:   ['RHEL','RMT1','RMT5','RTOE'],
                  reducedMarkerNames[2]:   ['LANK'],
                  reducedMarkerNames[3]:   ['RANK'],
                  reducedMarkerNames[4]:   ['LKNE','LKNI'],
                  reducedMarkerNames[5]:   ['RKNE','RKNI'],
                  reducedMarkerNames[6]:   ['LFWT','LBWT'],
                  reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                  reducedMarkerNames[8]:   ['RFWT','RBWT'],
                  reducedMarkerNames[9]:   ['C6','C7','T5','T10','BWT','STRN','CLAV','FWT'],
                  reducedMarkerNames[10]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                  reducedMarkerNames[11]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                  reducedMarkerNames[12]:  ['LOWR','LIWR'],
                  reducedMarkerNames[13]:  ['ROWR','RIWR'],
                  reducedMarkerNames[14]:  ['LELB','LIEL','LFRM'],
                  reducedMarkerNames[15]:  ['RELB','RIEL','RFRM'],
                  reducedMarkerNames[16]:  ['LSHO'],
                  reducedMarkerNames[17]:  ['RSHO','LSHO'],
                  reducedMarkerNames[18]:  ['RSHO'],
                  reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                  }
    #   removing nan joints
    for nan_joint_name,_ in max_nans_per_column[::3].items():
        nan_joint_name = nan_joint_name[:-2]
        for markerFullList in markersMap.values():
            if nan_joint_name in markerFullList:
                markerFullList.remove(nan_joint_name)
                break
    # removing many contiguous zeros joints
    for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
        if contiguous_zeros > 20:
            zero_joint_name = zero_joint_name[:-2]
            for markerFullList in markersMap.values():
                if zero_joint_name in markerFullList:
                    markerFullList.remove(zero_joint_name)
                    break
    originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    
    #   removing markers not present in the data markers
    for markerFullList in markersMap.values():
        for markerName in list(reversed(markerFullList)):
            if not markerName in originalDataColumns:
                markerFullList.remove(markerName)

    #   removing asymmetries across body
    for left_side_marker_indx,right_side_marker_indx in [(0,1),(4,5),(6,8),(10,11),(12,13),(14,15)]: # indices of every marker on the left side and his symmetric
        markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
        markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
        for elem in markersLeft:
            if not 'R'+elem[1:] in markersRight:
                markersLeft.remove(elem)
                if left_side_marker_indx == 6: 
                    markersMap[reducedMarkerNames[7]].remove(elem)
        for elem in markersRight:
            if not 'L'+elem[1:] in markersLeft:
                markersRight.remove(elem)
                if left_side_marker_indx == 6: 
                    markersMap[reducedMarkerNames[7]].remove(elem)

    for i,markerFullList in enumerate(markersMap.values()):
        if len(markerFullList) == 0:
            print(markersMap)
            raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
    return markersMap

In [40]:
reducedMarkerNames = ['left_foot',      # 0
                      'right_foot',     # 1          
                      'left_ank',       # 2      
                      'right_ank',      # 3                          
                      'left_knee',      # 4                          
                      'right_knee',     # 5                              
                      'left_hip',       # 6                          
                      'hip_central',    # 7                              
                      'right_hip',      # 8                          
                      'spine',          # 9                       
                      'left_hand',      # 10                           
                      'right_hand',     # 11                               
                      'left_wrist',     # 12                               
                      'right_wrist',    # 13                               
                      'left_elbow',     # 14                               
                      'right_elbow',    # 15                               
                      'left_shoulder',  # 16                               
                      'shoulder_center',# 17                                   
                      'right_shoulder', # 18                                   
                      'head']           # 19                       
fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

posTable = data.iloc[:,3:]

posTableX = posTable.iloc[:,::3]
posTableY = posTable.iloc[:,1::3]
posTableZ = posTable.iloc[:,2::3]

reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

#reducedPosTableX = (reducedPosTableX - reducedPosTableX.min(axis=None))/(reducedPosTableX.max(axis=None)-reducedPosTableX.min(axis=None))
#reducedPosTableY = (reducedPosTableY - reducedPosTableY.min(axis=None))/(reducedPosTableY.max(axis=None)-reducedPosTableY.min(axis=None))
#reducedPosTableZ = (reducedPosTableZ - reducedPosTableZ.min(axis=None))/(reducedPosTableZ.max(axis=None)-reducedPosTableZ.min(axis=None))

def xyz_tables_to_xyz_columns(tablesList):
    xTable,yTable,zTable = tablesList
    mergedTable = pd.DataFrame()
    for joint in range(xTable.shape[1]):
        mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
    return mergedTable

reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
reducedPosTable.head()

,left_foot_X,left_foot_Y,left_foot_Z,right_foot_X,right_foot_Y,right_foot_Z,left_ank_X,left_ank_Y,left_ank_Z,right_ank_X,...,left_shoulder_Z,shoulder_center_X,shoulder_center_Y,shoulder_center_Z,right_shoulder_X,right_shoulder_Y,right_shoulder_Z,head_X,head_Y,head_Z
0,0.599149,0.282866,0.281366,0.603479,0.282036,0.196394,0.626746,0.309066,0.291376,0.634263,...,0.310926,0.610581,0.753321,0.244585,0.613030,0.757138,0.178245,0.620697,0.841934,0.245584
1,0.599149,0.282867,0.281366,0.603481,0.282036,0.196393,0.626744,0.309068,0.291374,0.634298,...,0.310933,0.610508,0.753349,0.244600,0.612879,0.757172,0.178267,0.620695,0.841926,0.245566
2,0.599149,0.282866,0.281365,0.603482,0.282037,0.196392,0.626739,0.309063,0.291373,0.634445,...,0.310944,0.610425,0.753376,0.244616,0.612703,0.757201,0.178288,0.620691,0.841918,0.245548
3,0.599152,0.282863,0.281364,0.603482,0.282036,0.196392,0.626735,0.309062,0.291373,0.633893,...,0.310954,0.610346,0.753399,0.244627,0.612542,0.757220,0.178300,0.620685,0.841909,0.245532
4,0.599150,0.282868,0.281363,0.603481,0.282040,0.196393,0.626724,0.309069,0.291376,0.634368,...,0.310969,0.610267,0.753423,0.244642,0.612376,0.757248,0.178315,0.620681,0.841904,0.245516


## Writing to files final clean

In [41]:
import os

PROCESSED_PATH = 'data/reprocessed/'

dest_dir = os.path.join(PROCESSED_PATH,folder)
if not os.path.exists(dest_dir): 
    os.mkdir(dest_dir)
dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
if not os.path.isfile(dest_file_path):
    reducedPosTable.to_csv(dest_file_path,index=False)

## Plotting reduced markers

In [42]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
if sampleAnno < 45:
    posTableY = reducedPosTable.iloc[:,::3]
    posTableZ = reducedPosTable.iloc[:,1::3]
    posTableX = reducedPosTable.iloc[:,2::3]
else:
    posTableX = reducedPosTable.iloc[:,::3]
    posTableY = reducedPosTable.iloc[:,1::3]
    posTableZ = reducedPosTable.iloc[:,2::3]

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
jointsFrom = list(map(lambda x: x-1,jointsFrom))
jointsTo = list(map(lambda x: x-1,jointsTo))
edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
adjacencyMatrix = np.zeros((20,20),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    # Add edges based on the weight matrix
    for i in range(20):
        for j in range(i + 1, 20):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='gray', linestyle='-', linewidth=1)

    ax.set_xlim([-0.1,0.8])#minMax[0,0],minMax[1,0]])
    ax.set_ylim([-0.1,0.8])#minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    ax.set_title("Movement "+str(sampleAnno))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()

Using matplotlib backend: TkAgg


## Plotting full markers

In [79]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
if sampleAnno < 45:
    posTableY = posTable.iloc[:,::3]
    posTableZ = posTable.iloc[:,1::3]
    posTableX = posTable.iloc[:,2::3]
else:
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]
    

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    #ax.set_title("Movement "+str(picked))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


In [134]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

minMax[0,:] = [np.nanmin(posTableX.iloc[:1,:].values),np.nanmin(posTableY.iloc[:1,:].values),np.nanmin(posTableZ.iloc[:1,:].values)]
minMax[1,:] = [np.nanmax(posTableX.iloc[:1,:].values),np.nanmax(posTableY.iloc[:1,:].values),np.nanmax(posTableZ.iloc[:1,:].values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=0)
    
    for i, col_name in enumerate(posTableX.columns):
        ax.text(filteredX[i], filteredY[i], filteredZ[i], col_name[:-2], horizontalalignment='center',fontsize=6, color='black')

    ax.set_xticks([minMax[0,0],minMax[1,0]])
    ax.set_yticks([minMax[0,1],minMax[1,1]])
    ax.set_zticks([minMax[0,2],minMax[1,2]])
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    
    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


In [135]:
plt.close('all')

# All in One

In [47]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def view_sample_reduced(posTable, sample):
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,2::3]
    posTableZ = posTable.iloc[:,1::3]
    jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
    jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
    jointsFrom = list(map(lambda x: x-1,jointsFrom))
    jointsTo = list(map(lambda x: x-1,jointsTo))
    edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
    adjacencyMatrix = np.zeros((20,20),dtype=bool)
    adjacencyMatrix[edges[:,0],edges[:,1]] = True


    plt.close("all")
    fig = plt.figure()
    ax:plt.Axes = fig.add_subplot(111, projection='3d')


    minMax = np.zeros((2,3))
    minMax[0,:] = [posTableX.values.min(),posTableY.values.min(),posTableZ.values.min()]
    minMax[1,:] = [posTableX.values.max(),posTableY.values.max(),posTableZ.values.max()]

    print(minMax)

    # Set appropriate axis limits
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    # Set the window title
    fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

    # Set the initial time index
    time_index = 0
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'cyan']

    # Function to update the plot based on the slider value
    def update_plot(val):
        ax.cla()  # Clear the previous plot

        # Filter the data based on the current time index
        filteredX = posTableX.iloc[val]
        filteredY = posTableY.iloc[val]
        filteredZ = posTableZ.iloc[val]

        ax.scatter(filteredX,filteredY,filteredZ)

        # Add edges based on the weight matrix
        for i in range(20):
            for j in range(i + 1, 20):
                if adjacencyMatrix[i,j]:
                    ax.plot([filteredX[i], filteredX[j]],
                            [filteredY[i], filteredY[j]],
                            [filteredZ[i], filteredZ[j]],
                            color='gray', linestyle='-', linewidth=1)


        ax.set_xlim([minMax[0,0],minMax[1,0]])
        ax.set_ylim([minMax[0,1],minMax[1,1]])
        ax.set_zlim([minMax[0,2],minMax[1,2]])

        ax.set_xlabel('X', fontsize=12)
        ax.set_ylabel('Y', fontsize=12)
        ax.set_zlabel('Z', fontsize=12)
        ax.set_title("Movement "+str(sample))

        fig.canvas.draw_idle()

    # Create a slider widget
    slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
    maxValue = posTableX.shape[0]-1
    slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


    # Define a function to update the slider value with the mouse wheel
    def on_scroll(event):
        if event.button == 'down':
            if slider.val + slider.valstep*3 <= maxValue:
                slider.set_val(slider.val + slider.valstep*3)
        elif event.button == 'up':
            if slider.val - slider.valstep*3 >= 0:
                slider.set_val(slider.val - slider.valstep*3)


    # Connect the mouse wheel event to the function
    fig.canvas.mpl_connect('scroll_event', on_scroll)


    # Register the update_plot function with the slider widget
    slider.on_changed(update_plot)

    # Initial plot
    update_plot(time_index)

    # Show the plot
    plt.show(block=True)


Using matplotlib backend: TkAgg


In [51]:
import csv
def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break

import os
import pandas as pd
from tqdm.notebook import tqdm
ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
with open(ANNOTATIONS_PATH+'/annotationsVSingle.txt','r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    #print(len(annotations))
    
for sampleAnno in tqdm(range(len(annotations)),desc="Saving samples"):
    if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
        folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
    else:
        folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
    sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
    #print(sampleData,startSec,endSec)
    with open(sampleData,'r') as file:
        numMarkers, columnNames, readerBuffer = custom_header_reader(file)
        data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))
    #print(data['Time'])
    #print(sampleAnno, data['Time'].iloc[0])
    if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
        raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
    else:
        #print("\x1b[32mSTART AND END SECONDS CHECK CORRECT!\x1b[0m")
        pass


    import pandas as pd
    if '_A' in data.columns[3] or '_B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
    elif '61A' in data.columns[3] or '61B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
    if '_01' in data.columns[3]:
        data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
    if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
        mappings = pd.read_csv("data/raw/ExtendedMarkersToStandard.csv")
        mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
        data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]


    #print("Total NaNs")
    nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)
    color = '\x1b[32m' if len(nans) == 0 else '\x1b[31m'
    #print(color+str(nans)+'\x1b[0m '+ ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

    from itertools import groupby
    max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))
    #print("Max contiguous NaNs")
    max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)
    color = '\x1b[32m' if len(max_nans_per_column) == 0 else '\x1b[31m'
    #print(color+str(max_nans_per_column)+'\x1b[0m ' + ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

    #print('Num Zeros')
    zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
    color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
    #print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
    #list(zeros.index)

    # THIS IS A FIX FOR MISSING QTM AT DATA 58-59
    import numpy as np
    if len(zeros.index) > 15:
        for name_col,num_zeros in zeros.items():
            if num_zeros < 50:
                data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)


    from itertools import groupby
    max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
    #print("Max contiguous Zeros")
    max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)
    color = '\x1b[32m' if len(max_zeros_per_column) == 0 else '\x1b[31m'
    #print(color+str(max_zeros_per_column)+'\x1b[0m ' + ('\nNo Zeros' if len(max_zeros_per_column)==0 else '\nZeros found!'))


    def map_reduce_num_markers(reducedMarkerNames:list):
        if not 'ARIEL' in fullMarkerNames:
            pass
        markersMap = {reducedMarkerNames[0]:   ['RHEL','RMT1','RMT5','RTOE'],
                      reducedMarkerNames[1]:   ['LHEL','LMT1','LMT5','LTOE'],
                      reducedMarkerNames[2]:   ['RANK'],
                      reducedMarkerNames[3]:   ['LANK'],
                      reducedMarkerNames[4]:   ['RKNE','RKNI'],
                      reducedMarkerNames[5]:   ['LKNE','LKNI'],
                      reducedMarkerNames[6]:   ['RFWT','RBWT'],
                      reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                      reducedMarkerNames[8]:   ['LFWT','LBWT'],
                      reducedMarkerNames[9]:   ['C6','C7','T5','T10','BWT','STRN','CLAV','FWT'],
                      reducedMarkerNames[10]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                      reducedMarkerNames[11]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                      reducedMarkerNames[12]:  ['ROWR','RIWR'],
                      reducedMarkerNames[13]:  ['LOWR','LIWR'],
                      reducedMarkerNames[14]:  ['RELB','RIEL','RFRM'],
                      reducedMarkerNames[15]:  ['LELB','LIEL','LFRM'],
                      reducedMarkerNames[16]:  ['RSHO'],
                      reducedMarkerNames[17]:  ['RSHO','LSHO'],
                      reducedMarkerNames[18]:  ['LSHO'],
                      reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                      }
        
        # removing nan joints
        for nan_joint_name,_ in max_nans_per_column[::3].items():
            nan_joint_name = nan_joint_name[:-2]
            for markerFullList in markersMap.values():
                if nan_joint_name in markerFullList:
                    markerFullList.remove(nan_joint_name)
                    break
        # removing many contiguous zeros joints
        for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
            if contiguous_zeros > 20:
                zero_joint_name = zero_joint_name[:-2]
                for markerFullList in markersMap.values():
                    if zero_joint_name in markerFullList:
                        markerFullList.remove(zero_joint_name)
                        break
        originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

        # removing markers not present in the data markers
        for markerFullList in markersMap.values():
            for markerName in list(reversed(markerFullList)):
                if not markerName in originalDataColumns:
                    markerFullList.remove(markerName)

        # removing asymmetries across body
        for right_side_marker_indx,left_side_marker_indx in [(0,1),(2,3),(4,5),(6,8),(10,11),(12,13),(14,15),(16,18)]: # indices of every marker on the left side and his symmetric
            markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
            markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
            for elem in markersLeft:
                if not 'R'+elem[1:] in markersRight:
                    markersLeft.remove(elem)
                    if left_side_marker_indx == 6 or left_side_marker_indx == 16: 
                        markersMap[reducedMarkerNames[left_side_marker_indx+1]].remove(elem)
            for elem in markersRight:
                if not 'L'+elem[1:] in markersLeft:
                    markersRight.remove(elem)
                    if right_side_marker_indx == 8 or right_side_marker_indx == 18: 
                        markersMap[reducedMarkerNames[right_side_marker_indx-1]].remove(elem)

        for i,markerFullList in enumerate(markersMap.values()):
            if len(markerFullList) == 0:
                print(markersMap)
                raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
        return markersMap

    reducedMarkerNames = ['right_foot',      # 1
                          'left_foot',       # 2        
                          'right_ank',       # 3      
                          'left_ank',        # 4                          
                          'right_knee',      # 5                          
                          'left_knee',       # 6                              
                          'right_hip',       # 7                          
                          'hip_central',     # 8                              
                          'left_hip',        # 9                          
                          'spine',           # 10                       
                          'right_hand',      # 11                           
                          'left_hand',       # 12                               
                          'right_wrist',     # 13                               
                          'left_wrist',      # 14                               
                          'right_elbow',     # 15                               
                          'left_elbow',      # 16                               
                          'right_shoulder',  # 17                               
                          'shoulder_center', # 18                                   
                          'left_shoulder',   # 19                                   
                          'head']            # 20                       
    fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
    reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

    posTable = data.iloc[:,3:]

    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]

    reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
    reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
    reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

    #reducedPosTableX = (reducedPosTableX - reducedPosTableX.min(axis=None))/(reducedPosTableX.max(axis=None)-reducedPosTableX.min(axis=None))
    #reducedPosTableY = (reducedPosTableY - reducedPosTableY.min(axis=None))/(reducedPosTableY.max(axis=None)-reducedPosTableY.min(axis=None))
    #reducedPosTableZ = (reducedPosTableZ - reducedPosTableZ.min(axis=None))/(reducedPosTableZ.max(axis=None)-reducedPosTableZ.min(axis=None))

    def xyz_tables_to_xyz_columns(tablesList):
        xTable,yTable,zTable = tablesList
        mergedTable = pd.DataFrame()
        for joint in range(xTable.shape[1]):
            mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
        return mergedTable

    reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
    reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
    #view_sample_reduced(reducedPosTable,sampleAnno)

    import os

    PROCESSED_PATH = 'data/reprocessed/'

    dest_dir = os.path.join(PROCESSED_PATH,folder)
    if not os.path.exists(dest_dir): 
        os.mkdir(dest_dir)
    dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
    if not os.path.isfile(dest_file_path) or True:
        reducedPosTable.to_csv(dest_file_path,index=False)

Saving samples:   0%|          | 0/60 [00:00<?, ?it/s]